# Instacart Market Basket Analysis

The goal of this project is to gain comprehensive insights into customer behavior, product performance, and purchasing patterns using transactional data. The project will be divided into several key stages, including data exploration and preprocessing, exploratory data analysis (EDA), and business questions and analysis. The following are the detailed requirements:

## 1. Data Exploration and Preprocessing
a. Load the Data:
- Load the CSV files into pandas DataFrames.

In [7]:
import pandas as pd

# Load the CSV files into pandas DataFrames
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
order_products_prior = pd.read_csv('data/order_products__prior.csv')
order_products_train = pd.read_csv('data/order_products__train.csv')
aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')

- Check for missing values and data types.

In [2]:
# Create a list of DataFrames
dfs = [orders, products, order_products_prior, order_products_train, aisles, departments]
df_names = ['Orders', 'Products', 'Order Products Prior', 'Order Products Train', 'Aisles', 'Departments']

# Create a dictionary to hold the missing values information
missing_values_dict = {'DataFrame': [], 'Column': [], 'Missing Values': []}

# Check for missing values in each DataFrame
for df, name in zip(dfs, df_names):
    missing_values = df.isnull().sum()
    for column, missing in missing_values.items():
        missing_values_dict['DataFrame'].append(name)
        missing_values_dict['Column'].append(column)
        missing_values_dict['Missing Values'].append(missing)

# Convert the dictionary to a DataFrame
missing_values_df = pd.DataFrame(missing_values_dict)

In [3]:
missing_values_df

,DataFrame,Column,Missing Values
0,Orders,order_id,0
1,Orders,user_id,0
2,Orders,eval_set,0
3,Orders,order_number,0
4,Orders,order_dow,0
5,Orders,order_hour_of_day,0
6,Orders,days_since_prior_order,206209
7,Products,product_id,0
8,Products,product_name,0
9,Products,aisle_id,0


b. Data Cleaning:
- Handle missing values.

In [4]:
# Handling missing values
# Here we'll fill missing values with placeholders
orders.fillna({'days_since_prior_order': 0}, inplace=True)

- Merge DataFrames as needed (e.g., joining order_products with products).

In [5]:
#concat train order and prior orders
order_products_all = pd.concat([order_products_prior, order_products_train]).sort_values(by=['order_id'])

#merge with rest
order_products_all = pd.merge(left = order_products_all, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
order_products_all = pd.merge(left = order_products_all, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id','order_id','product_id','aisle_id','department_id','add_to_cart_order',
 'reordered','product_name','aisle','department','eval_set','order_number','order_dow','order_hour_of_day',
 'days_since_prior_order']

order_products_all = order_products_all[col_order]

In [6]:
order_products_all.head()

,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,112108,1,43633,95,15,5,1,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,train,4,4,10,9.0
1,112108,1,49302,120,16,1,1,Bulgarian Yogurt,yogurt,dairy eggs,train,4,4,10,9.0
2,112108,1,49683,83,4,4,0,Cucumber Kirby,fresh vegetables,produce,train,4,4,10,9.0
3,112108,1,22035,21,16,8,1,Organic Whole String Cheese,packaged cheese,dairy eggs,train,4,4,10,9.0
4,112108,1,10246,83,4,3,0,Organic Celery Hearts,fresh vegetables,produce,train,4,4,10,9.0


c. Basic Descriptive Statistics:
- Calculate summary statistics for numerical features.

In [6]:
order_products_all['user_id'].nunique()

206209

In [9]:
# Summary statistics for numerical features in the order_products_all DataFrame
print("\nSummary Statistics for All Order Products DataFrame:")
order_products_all.describe()


Summary Statistics for All Order Products DataFrame:


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07
mean,1.029444e+05,1.710566e+06,2.557551e+04,7.121799e+01,9.918544e+00,8.367738e+00,5.900617e-01,1.713998e+01,2.737285e+00,1.343123e+01,1.066586e+01
std,5.946733e+04,9.874008e+05,1.409770e+04,3.819898e+01,6.281655e+00,7.139540e+00,4.918220e-01,1.749829e+01,2.093296e+00,4.246149e+00,9.081234e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.143500e+04,8.554130e+05,1.351900e+04,3.100000e+01,4.000000e+00,3.000000e+00,0.000000e+00,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.026260e+05,1.710660e+06,2.525600e+04,8.300000e+01,9.000000e+00,6.000000e+00,1.000000e+00,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,1.544120e+05,2.565587e+06,3.793500e+04,1.070000e+02,1.600000e+01,1.100000e+01,1.000000e+00,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,2.062090e+05,3.421083e+06,4.968800e+04,1.340000e+02,2.100000e+01,1.450000e+02,1.000000e+00,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [10]:
order_products_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 int64  
 1   order_id                int64  
 2   product_id              int64  
 3   aisle_id                int64  
 4   department_id           int64  
 5   add_to_cart_order       int64  
 6   reordered               int64  
 7   product_name            object 
 8   aisle                   object 
 9   department              object 
 10  eval_set                object 
 11  order_number            int64  
 12  order_dow               int64  
 13  order_hour_of_day       int64  
 14  days_since_prior_order  float64
dtypes: float64(1), int64(10), object(4)
memory usage: 3.8+ GB


In [11]:
order_products_all.describe(include='object')

,product_name,aisle,department,eval_set
count,33819106,33819106,33819106,33819106
unique,49685,134,21,2
top,Banana,fresh fruits,produce,prior
freq,491291,3792661,9888378,32434489


- Examine the distribution of categorical features.

In [18]:
# Distribution of categorical features in the order_products_all_merged DataFrame
print("\nDistribution of product_id in All Order Products Merged DataFrame:")
order_products_all['product_name'].value_counts().head()


Distribution of product_id in All Order Products Merged DataFrame:


product_name
Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Name: count, dtype: int64

In [19]:
print("\nDistribution of aisle_id in All Order Products Merged DataFrame:")
order_products_all['aisle_id'].value_counts().head()


Distribution of aisle_id in All Order Products Merged DataFrame:


aisle_id
24     3792661
83     3568630
123    1843806
120    1507583
21     1021462
Name: count, dtype: int64

In [20]:
print("\nDistribution of department_id in All Order Products Merged DataFrame:")
order_products_all['department_id'].value_counts().head()


Distribution of department_id in All Order Products Merged DataFrame:


department_id
4     9888378
16    5631067
19    3006412
7     2804175
1     2336858
Name: count, dtype: int64

## 2. Exploratory Data Analysis (EDA)
a. Customer Behavior:
- Average number of orders per user.

In [9]:
# Calculate the number of orders per user
orders_per_user = orders.groupby('user_id').size()

# Calculate the average number of orders per user
average_orders_per_user = orders_per_user.mean()
print(f"Average number of orders per user: {average_orders_per_user:.2f}")


Average number of orders per user: 16.59


- Average time between orders for each user.

In [10]:
average_days_between_orders = orders.groupby('user_id')['days_since_prior_order'].mean()
average_days_between_orders

user_id
1         17.272727
2         15.200000
3         11.076923
4         14.166667
5          9.200000
            ...    
206205    12.500000
206206     3.661765
206207    13.470588
206208     7.220000
206209    17.142857
Name: days_since_prior_order, Length: 206209, dtype: float64

- Number of orders placed by each customer.

In [11]:
# Calculate the number of orders placed by each customer
orders_count_per_customer = orders.groupby('user_id').size()

# Display the number of orders for the first few customers
print("Number of orders placed by each customer (sample):")
print(orders_count_per_customer.head())

Number of orders placed by each customer (sample):
user_id
1    11
2    15
3    13
4     6
5     5
dtype: int64


- Customer segments based on purchase frequency.

In [12]:
# Define customer segments based on the number of orders
order_counts = orders_count_per_customer

# Create segments based on quantiles
segments = pd.qcut(order_counts, q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Merge the segments with the customer data
customer_segments = pd.DataFrame({
    'user_id': order_counts.index,
    'order_count': order_counts.values,
    'segment': segments
})

print("\nCustomer Segments based on Purchase Frequency:")
customer_segments.head()


Customer Segments based on Purchase Frequency:


,user_id,order_count,segment
user_id,,,
1,1,11,High
2,2,15,High
3,3,13,High
4,4,6,Low
5,5,5,Low


b. Product Analysis:
- Identify most popular products by frequency.

In [13]:
# Count the frequency of each product_id
product_frequency = order_products_all['product_id'].value_counts()

product_frequency_df = product_frequency.reset_index()
product_frequency_df.columns = ['product_id', 'frequency']

# Merge with products to get product names
product_frequency_df = product_frequency_df.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Sort by frequency and display the top products
top_products = product_frequency_df.sort_values(by='frequency', ascending=False)
print("\nMost Popular Products by Frequency:")
print(top_products.head())


Most Popular Products by Frequency:
   product_id  frequency            product_name
0       24852     491291                  Banana
1       13176     394930  Bag of Organic Bananas
2       21137     275577    Organic Strawberries
3       21903     251705    Organic Baby Spinach
4       47209     220877    Organic Hass Avocado


- Determine average order size (number of items per order).

In [14]:
# Calculate the number of items per order
order_sizes = order_products_all.groupby('order_id').size()

# Calculate the average order size
average_order_size = order_sizes.mean()
print(f"\nAverage Order Size (Number of Items per Order): {average_order_size:.2f}")


Average Order Size (Number of Items per Order): 10.11


c. Temporal Patterns:
- Analyze orders by day of the week and hour of the day.

In [16]:
orders_by_dow = order_products_all['order_dow'].value_counts()
orders_by_hour = order_products_all['order_hour_of_day'].value_counts()

orders_by_dow

order_dow
0    6533692
1    5871834
6    4707583
5    4386443
2    4378360
3    3998498
4    3942696
Name: count, dtype: int64

In [17]:
orders_by_hour

order_hour_of_day
10    2874905
11    2852701
14    2810918
15    2780731
13    2778054
12    2732599
16    2647695
9     2550569
17    2186409
8     1787359
18    1714445
19    1317576
20    1017958
7      928239
21     831183
22     662053
23     419585
6      302642
0      228031
1      121412
5       91909
2       72660
4       55714
3       53759
Name: count, dtype: int64

In [18]:
# Determine if the order was on a weekend or weekday
order_products_all['is_weekend'] = order_products_all['order_dow'].apply(lambda x: x in [0, 6])  # 0 = Sunday, 6 = Saturday

In [19]:
product_sales_by_day = order_products_all.groupby(['product_id', 'product_name', 'is_weekend'])['order_id'].count().reset_index()
product_sales_by_day

,product_id,product_name,is_weekend,order_id
0,1,Chocolate Sandwich Cookies,False,1597
1,1,Chocolate Sandwich Cookies,True,331
2,2,All-Seasons Salt,False,58
3,2,All-Seasons Salt,True,36
4,3,Robust Golden Unsweetened Oolong Tea,False,176
...,...,...,...,...
98183,49686,Artisan Baguette,True,46
98184,49687,Smartblend Healthy Metabolism Dry Cat Food,False,12
98185,49687,Smartblend Healthy Metabolism Dry Cat Food,True,2
98186,49688,Fresh Foaming Cleanser,False,61


In [20]:
popular_products_weekday = product_sales_by_day[product_sales_by_day['is_weekend'] == False].sort_values('order_id', ascending=False)
popular_products_weekday

,product_id,product_name,is_weekend,order_id
49129,24852,Banana,False,323814
26045,13176,Bag of Organic Bananas,False,268435
41806,21137,Organic Strawberries,False,180606
43318,21903,Organic Baby Spinach,False,157547
93282,47209,Organic Hass Avocado,False,145446
...,...,...,...,...
89668,45378,Miss Treated Conditioner,False,1
22745,11508,FiberSmart 30 Day Supply,False,1
89998,45547,Grapefruit Rejuvenating Body Wash,False,1
90018,45557,Healing + Vitamin E Conditioner,False,1


In [21]:
popular_products_weekend = product_sales_by_day[product_sales_by_day['is_weekend'] == True].sort_values('order_id', ascending=False)
popular_products_weekend

,product_id,product_name,is_weekend,order_id
49130,24852,Banana,True,167477
26046,13176,Bag of Organic Bananas,True,126495
41807,21137,Organic Strawberries,True,94971
43319,21903,Organic Baby Spinach,True,94158
93283,47209,Organic Hass Avocado,True,75431
...,...,...,...,...
33703,17049,Teething Gel,True,1
33857,17126,Classic Frankfurters,True,1
33859,17127,Organic Chamomile Citrus Tea,True,1
33879,17138,Positively Pancakes,True,1


d. Basket Analysis:
- Identify most frequently co-purchased items.

In [70]:
chunk = order_products_all.head(100000)

In [75]:
# Create a new DataFrame with all possible pairs within each order
merged = pd.merge(chunk, chunk, on='order_id')

# Filter out self-pairs and sort product pairs
merged = merged[merged['product_id_x'] != merged['product_id_y']]
merged['product_pair'] = merged.apply(lambda row: tuple(sorted((row['product_id_x'], row['product_id_y']))), axis=1)

In [117]:
# Group by product pairs and count occurrences
pair_counts = merged.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs = pair_counts.sort_values('frequency', ascending=False)

copurchased_pairs.head(10)

,product_pair,frequency
254956,"(13176, 47209)",382
375023,"(21137, 24852)",376
252320,"(13176, 21137)",346
252399,"(13176, 21903)",316
431071,"(24852, 47766)",312
388241,"(21903, 24852)",300
309350,"(16797, 24852)",272
253008,"(13176, 27966)",258
374813,"(21137, 21903)",252
376749,"(21137, 47209)",234


- Products often bought together on weekends vs. weekdays.

In [88]:
merged_weekend = merged[merged['is_weekend_x'] & merged['is_weekend_y']]
pair_counts_weekend = merged_weekend.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs_weekend = pair_counts_weekend.sort_values('frequency', ascending=False)

copurchased_pairs_weekend.head()


,product_pair,frequency
97643,"(13176, 47209)",154
142211,"(21137, 24852)",136
96321,"(13176, 21137)",124
96357,"(13176, 21903)",122
147445,"(21903, 24852)",120


In [90]:
merged_weekday = merged[~(merged['is_weekend_x'] & merged['is_weekend_y'])]
pair_counts_weekday = merged_weekday.groupby('product_pair').size().reset_index(name='frequency')

# Sort by frequency and return top 10
copurchased_pairs_weekday = pair_counts_weekday.sort_values('frequency', ascending=False)

copurchased_pairs_weekday.head()


,product_pair,frequency
253195,"(21137, 24852)",240
291696,"(24852, 47766)",228
171259,"(13176, 47209)",228
169205,"(13176, 21137)",222
169271,"(13176, 21903)",194


## 3. Business Questions and Analysis
a. Popular Products:
- Analyze sales distribution of top-selling products.

In [96]:
order_products_all['product_id'].nunique()

49685

In [113]:
product_counts = order_products_all.groupby(['product_id', 'product_name']).size().sort_values(ascending=False).reset_index(name='count')
product_counts

,product_id,product_name,count
0,24852,Banana,491291
1,13176,Bag of Organic Bananas,394930
2,21137,Organic Strawberries,275577
3,21903,Organic Baby Spinach,251705
4,47209,Organic Hass Avocado,220877
...,...,...,...
49680,2769,Pappardelle Nests Pasta,1
49681,35542,Lindor Peppermint White Chocolate Truffles,1
49682,36079,Pumpkin Spice Almond Nog,1
49683,7626,Brut Prosecco,1


- Identify top 5 products commonly added to the cart first.

In [116]:
# Filter for products added first (add_to_cart_order == 1)
first_added_products = order_products_all[order_products_all['add_to_cart_order'] == 1]

# Count the frequency of each product
product_counts = first_added_products.groupby(['product_id', 'product_name']).size().reset_index(name='frequency')

# Sort by frequency and return top 5
top_5_products = product_counts.sort_values('frequency', ascending=False).head(5)
top_5_products

,product_id,product_name,frequency
21084,24852,Banana,115521
11142,13176,Bag of Organic Bananas,82877
23620,27845,Organic Whole Milk,32071
17909,21137,Organic Strawberries,28875
40105,47209,Organic Hass Avocado,24913


b. Reorder Behavior:
- Analyze the reordered column to understand repeat purchase behavior.
- Products reordered the most.

In [118]:
# Count reordered products
reordered_products = order_products_all[order_products_all['reordered'] == 1]['product_id'].nunique()
total_products = order_products_all['product_id'].nunique()

# Calculate reorder rate
reorder_rate = reordered_products / total_products

print("Reorder rate:", reorder_rate)

# Identify top reordered products
top_reordered_products = order_products_all[order_products_all['reordered'] == 1].groupby('product_id').size().reset_index(name='reorder_count').sort_values('reorder_count', ascending=False)

top_reordered_products.head(5)

Reorder rate: 0.9178424071651404


,product_id,reorder_count
22832,24852,415166
12083,13176,329275
19415,21137,214448
20110,21903,194939
43334,47209,176173


- Reorder behavior based on day of the week and days since prior order.

In [133]:
# Filter for reordered products
reordered_data = order_products_all[order_products_all['reordered'] == 1]

# Calculate reorder rate by day of week
reordered_by_day = reordered_data.groupby(reordered_data['order_dow'])['product_id'].nunique() / reordered_data['product_id'].nunique()

# Calculate reorder rate by days since prior order
reordered_by_days_since_prior_order = reordered_data.groupby('days_since_prior_order')['product_id'].nunique() / reordered_data['product_id'].nunique()


In [134]:
reordered_by_day.sort_values(ascending=False)

order_dow
0    0.815407
1    0.808412
6    0.798083
5    0.794246
2    0.785738
4    0.780914
3    0.779773
Name: product_id, dtype: float64

In [135]:
reordered_by_days_since_prior_order.sort_values(ascending=False)

days_since_prior_order
7.0     0.746420
30.0    0.711927
6.0     0.710392
5.0     0.679977
8.0     0.677762
4.0     0.668355
3.0     0.647633
9.0     0.615025
2.0     0.610552
14.0    0.589238
10.0    0.579129
13.0    0.561761
1.0     0.554503
11.0    0.551411
12.0    0.548385
0.0     0.528189
15.0    0.519725
16.0    0.462206
21.0    0.447054
17.0    0.433590
20.0    0.432165
18.0    0.422319
19.0    0.410105
22.0    0.393834
28.0    0.363638
23.0    0.347433
27.0    0.331711
24.0    0.331426
25.0    0.319869
26.0    0.313861
29.0    0.313181
Name: product_id, dtype: float64

- How the number of items in the cart impacts the likelihood of reordering.

In [136]:
# Group by order_id and count items in the cart
order_data = order_products_all.groupby('order_id').size().reset_index(name='cart_size')

# Merge with original DataFrame
merged_data = pd.merge(order_products_all, order_data, on='order_id')

# Calculate reorder rate by cart size
reordered_by_cart_size = merged_data[merged_data['reordered'] == 1].groupby('cart_size')['product_id'].nunique() / merged_data['product_id'].nunique()


In [138]:
reordered_by_cart_size.sort_values(ascending=False)

cart_size
9      0.591305
10     0.590319
8      0.587944
7      0.582309
11     0.582127
         ...   
145    0.001610
90     0.001469
105    0.001026
114    0.000845
127    0.000081
Name: product_id, Length: 113, dtype: float64

d. Department and Aisle Analysis:
- Best-selling department and aisle breakdown.

In [161]:
# Calculate total sales by department and aisle
sales_data = order_products_all.groupby(['department', 'aisle']).size().reset_index(name='total_sales')


department
produce       9888378
dairy eggs    5631067
snacks        3006412
beverages     2804175
frozen        2336858
Name: total_sales, dtype: int64

In [163]:
# Identify top-selling departments and aisles
top_departments = sales_data.groupby('department')['total_sales'].sum().nlargest(5)
top_departments

department
produce       9888378
dairy eggs    5631067
snacks        3006412
beverages     2804175
frozen        2336858
Name: total_sales, dtype: int64

In [164]:
top_aisles = sales_data.groupby('aisle')['total_sales'].sum().nlargest(5)
top_aisles

aisle
fresh fruits                  3792661
fresh vegetables              3568630
packaged vegetables fruits    1843806
yogurt                        1507583
packaged cheese               1021462
Name: total_sales, dtype: int64

- What is the “produce” department? Break it down by aisle.

In [165]:
# Filter for products in the "produce" department
produce_data = order_products_all[order_products_all['department'] == 'produce']

# Group by aisle and count sales
produce_by_aisle = produce_data.groupby('aisle').size().reset_index(name='total_sales')

# Sort by total sales
produce_by_aisle = produce_by_aisle.sort_values('total_sales', ascending=False)

produce_by_aisle

,aisle,total_sales
0,fresh fruits,3792661
2,fresh vegetables,3568630
4,packaged vegetables fruits,1843806
1,fresh herbs,393793
3,packaged produce,289488


- Differences in purchasing behavior based on different departments or aisles.

In [177]:
# Calculate average order size by department and aisle
average_order_size = order_products_all.groupby(['department', 'aisle']).size().reset_index(name='order_size')
average_order_size

,department,aisle,order_size
0,alcohol,beers coolers,50496
1,alcohol,red wines,36424
2,alcohol,specialty wines champagnes,11659
3,alcohol,spirits,29069
4,alcohol,white wines,31646
...,...,...,...
129,snacks,ice cream toppings,11706
130,snacks,mint gum,23626
131,snacks,nuts seeds dried fruit,319019
132,snacks,popcorn jerky,170441


In [179]:
# Calculate reorder rate by department and aisle
reordered_data = order_products_all[order_products_all['reordered'] == 1]
reordered_data

,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,is_weekend
0,112108,1,43633,95,15,5,1,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,train,4,4,10,9.0,False
1,112108,1,49302,120,16,1,1,Bulgarian Yogurt,yogurt,dairy eggs,train,4,4,10,9.0,False
3,112108,1,22035,21,16,8,1,Organic Whole String Cheese,packaged cheese,dairy eggs,train,4,4,10,9.0,False
5,112108,1,11109,108,16,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,other creams cheeses,dairy eggs,train,4,4,10,9.0,False
8,202279,2,33120,86,16,1,1,Organic Egg Whites,eggs,dairy eggs,prior,3,5,9,8.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33819095,175185,3421082,43352,32,4,4,1,Raspberries,packaged produce,produce,prior,23,2,18,4.0,False
33819097,25247,3421083,39678,74,17,6,1,Free & Clear Natural Dishwasher Detergent,dish detergents,household,prior,24,2,6,21.0,False
33819100,25247,3421083,24852,24,4,9,1,Banana,fresh fruits,produce,prior,24,2,6,21.0,False
33819101,25247,3421083,18176,92,18,4,1,Organic Strawberry Yogurt & Fruit Snack,baby food formula,babies,prior,24,2,6,21.0,False


In [180]:
reorder_rate = reordered_data.groupby(['department', 'aisle']).size() / order_products_all.groupby(['department', 'aisle']).size()
reorder_rate

department  aisle                     
alcohol     beers coolers                 0.566916
            red wines                     0.550022
            specialty wines champagnes    0.487263
            spirits                       0.572844
            white wines                   0.631928
                                            ...   
snacks      ice cream toppings            0.268239
            mint gum                      0.561373
            nuts seeds dried fruit        0.519358
            popcorn jerky                 0.592129
            trail mix snack mix           0.620867
Length: 134, dtype: float64

In [181]:
# Analyze customer demographics by department and aisle
customer_demographics = order_products_all.groupby(['department', 'aisle']).agg({'user_id': 'nunique'})
customer_demographics

user_id
department aisle                              
alcohol    beers coolers                  8278
           red wines                      7258
           specialty wines champagnes     3898
           spirits                        5837
           white wines                    6514
...                                        ...
snacks     ice cream toppings             6977
           mint gum                       6949
           nuts seeds dried fruit        78931
           popcorn jerky                 46523
           trail mix snack mix           12184

[134 rows x 1 columns]

## Date-based Analytics

In [208]:
orders_chunk = orders.head(1000)

In [276]:
# Create a starting date for simulation
start_date = pd.to_datetime('2023-08-20')  # Choose a reference date

# Sort orders by user_id and order_number
orders_sorted = orders_chunk.sort_values(by=['user_id', 'order_number'])

# Initialize the simulated order_date column
orders_sorted['simulated_date'] = start_date

In [277]:
# Generate simulated dates based on days_since_prior_order
for user_id, group in orders_sorted.head(1000).groupby('user_id'):
    group['simulated_date'] = start_date + pd.to_timedelta(group['days_since_prior_order'].fillna(0).cumsum(), unit='d')
    orders_sorted.loc[group.index, 'simulated_date'] = group['simulated_date']

In [278]:
# Ensure that the simulated dates are updated correctly
orders_sorted['simulated_date'] = pd.to_datetime(orders_sorted['simulated_date'])

In [279]:
orders_sorted.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,simulated_date
0,2539329,1,prior,1,2,8,0.0,2023-08-20
1,2398795,1,prior,2,3,7,15.0,2023-09-04
2,473747,1,prior,3,3,12,21.0,2023-09-25
3,2254736,1,prior,4,4,7,29.0,2023-10-24
4,431534,1,prior,5,4,15,28.0,2023-11-21


- Explore seasonal trends or patterns in purchasing behavior.

In [280]:
# Calculate quarterly sales
sales_by_quarter = orders_sorted.groupby(pd.Grouper(key='simulated_date', freq='QE'))['order_id'].count()
sales_by_quarter

simulated_date
2023-09-30    264
2023-12-31    332
2024-03-31    221
2024-06-30    136
2024-09-30     47
Freq: QE-DEC, Name: order_id, dtype: int64

- Months with higher order volumes.

In [281]:
# Add a month column for aggregation
orders_sorted['month'] = orders_sorted['simulated_date'].dt.to_period('M')

# Calculate monthly order volumes
monthly_order_volumes = orders_sorted.groupby('month').size()

print("\nMonthly Order Volumes:")
print(monthly_order_volumes)


Monthly Order Volumes:
month
2023-08    109
2023-09    155
2023-10    126
2023-11    116
2023-12     90
2024-01     75
2024-02     67
2024-03     79
2024-04     56
2024-05     44
2024-06     36
2024-07     29
2024-08     18
Freq: M, dtype: int64


In [282]:
orders_sorted['simulated_date'].describe()

count                             1000
mean     2023-12-21 20:51:21.599999744
min                2023-08-20 00:00:00
25%                2023-09-28 18:00:00
50%                2023-11-28 00:00:00
75%                2024-03-03 06:00:00
max                2024-08-18 00:00:00
Name: simulated_date, dtype: object

- Identify customers who haven’t placed an order in the last 30 days.

In [283]:
import datetime

# Calculate the current date
current_date = datetime.datetime.now()

# Identify customers who have placed orders in the last 30 days
recent_customers = orders_sorted[orders_sorted['simulated_date'] >= current_date - pd.Timedelta(days=30)]['user_id'].unique()

# Find all customers
all_customers = orders_sorted['user_id'].unique()

# Identify inactive customers
inactive_customers = set(all_customers) - set(recent_customers)

# Create DataFrame for inactive customers
inactive_customers_df = pd.DataFrame(list(inactive_customers), columns=['user_id'])

inactive_customers_df


,user_id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


- Percentage of customers who have churned in the past quarter.

In [284]:
# This example assumes a quarter is 3 months
quarter_start_date = current_date - pd.DateOffset(months=3)

# Identify customers who have placed orders in the past quarter
recent_customers = orders_sorted[
    (orders_sorted['simulated_date'] >= quarter_start_date) & 
    (orders_sorted['simulated_date'] <= current_date)
]['user_id'].unique()

# Find all customers
all_customers = orders_sorted['user_id'].unique()

# Identify churned customers
churned_customers = set(all_customers) - set(recent_customers)

# Calculate the percentage of churned customers
churned_customers_count = len(churned_customers)
total_customers_count = len(all_customers)
churn_rate_percentage = (churned_customers_count / total_customers_count) * 100

churn_rate_percentage


85.9375

## Thank You!